<a href="https://colab.research.google.com/github/rahiakela/machine-learning-research-and-practice/blob/main/hands-on-machine-learning-with-scikit-learn-keras-and-tensorflow/12-custom-models-and-training-with-tensorflow/01_tensorflow_like_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using TensorFlow like NumPy

In fact, 95% of the use cases you will encounter will not require anything other than `tf.keras` and `tf.data`.

But now it’s time to dive deeper into TensorFlow
and take a look at its lower-level Python API. This will be useful when you need extra
control to write custom loss functions, custom metrics, layers, models, initializers,
regularizers, weight constraints, and more. 

You may even need to fully control the
training loop itself, for example to apply special transformations or constraints to the
gradients (beyond just clipping them) or to use multiple optimizers for different parts
of the network.

TensorFlow’s API revolves around tensors, which flow from operation to operation—hence the name TensorFlow.

A tensor is very similar to a NumPy ndarray: it is usually
a multidimensional array, but it can also hold a scalar (a simple value, such as 42).
These tensors will be important when we create custom cost functions, custom metrics,
custom layers, and more, so let’s see how to create and manipulate them.



##Setup

In [1]:
import sys
import sklearn
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import time

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## Tensors and Operations

In [2]:
# a tensor representing a matrix with two rows and three columns of floats
tf.constant([
   [1., 2., 3.],
   [4., 5., 6.]          
])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
# a tensor representing a scalar
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [4]:
# a tensor has a shape and a data type
t = tf.constant([
   [1., 2., 3.],
   [4., 5., 6.]              
])
t.shape

TensorShape([2, 3])

In [5]:
t.dtype

tf.float32

In [6]:
# let's do indexing
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[:, :-1]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 2.],
       [4., 5.]], dtype=float32)>

In [8]:
t[..., 1, tf.newaxis] # get second column value as column-vector

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [9]:
t[..., 2, tf.newaxis] # get last column value as column-vector

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[3.],
       [6.]], dtype=float32)>

In [11]:
t[0, ..., tf.newaxis] # get first row value as row-vector

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[1.],
       [2.],
       [3.]], dtype=float32)>

In [12]:
t[1, ..., tf.newaxis] # get second row value as row-vector

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[4.],
       [5.],
       [6.]], dtype=float32)>

In [13]:
# let's do some tensor operations
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [15]:
t * 2

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [14]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [16]:
t ** 2

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [17]:
# let's calculate dot-product
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [19]:
tf.matmul(t, tf.transpose(t))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [35]:
# we must use tf.transpose(t)
try:
  tf.matmul(t, t.T)
except AttributeError as ex:
  print(ex)


        'EagerTensor' object has no attribute 'T'.
        If you are looking for numpy-related methods, please run the following:
        from tensorflow.python.ops.numpy_ops import np_config
        np_config.enable_numpy_behavior()


##Keras’ Low-Level API

The Keras API has its own low-level API, located in `keras.backend`.

In [22]:
K = keras.backend

K.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [23]:
t @ K.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [25]:
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

##Tensors and NumPy

Tensors play nice with NumPy: you can create a tensor from a NumPy array, and vice versa.

In [26]:
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [27]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [28]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [29]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

##Type Conversions

Type conversions can significantly hurt performance, and they can easily go unnoticed
when they are done automatically. 

To avoid this, TensorFlow does not perform any type conversions automatically:it just raises an exception if you try to execute an
operation on tensors with incompatible types.

In [30]:
try:
  tf.constant(2.0) + tf.constant(40)
except tf.errors.InvalidArgumentError as ex:
  print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]


In [36]:
try:
  tf.constant(2.0) + tf.constant(40., dtype=tf.float64)
except tf.errors.InvalidArgumentError as ex:
  print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]


In [37]:
# so, we need to convert types
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

##Variables

The `tf.Tensor` values we’ve seen so far are immutable: you cannot modify them. This
means that we cannot use regular tensors to implement weights in a neural network,
since they need to be tweaked by backpropagation. 

Plus, other parameters may also
need to change over time (e.g., a momentum optimizer keeps track of past gradients).

What we need is a `tf.Variable`:

In [38]:
v = tf.Variable([
   [1., 2., 3.],
   [4., 5., 6.]              
])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [39]:
# it can also be modified in place
v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [40]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [42]:
v[0, 2].assign(60)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42., 60.],
       [ 8., 10., 12.]], dtype=float32)>

In [43]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [44]:
v[:, 0].assign([20., 80.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[20., 42.,  0.],
       [80., 10.,  1.]], dtype=float32)>

In [52]:
v[0, 1].assign(6.)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[20.,  6.,  0.],
       [80., 10.,  1.]], dtype=float32)>

In [55]:
# but direct item assignment will not work
try:
  v[1] = [7., 8., 9.]
except TypeError as ex:
  print(ex)

'ResourceVariable' object does not support item assignment


So, we can do it by using the `scatter_update()` or `scatter_nd_update()` methods.

In [54]:
v.scatter_nd_update(indices=[[0, 0], [1, 2]], 
                    updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,   6.,   0.],
       [ 80.,  10., 200.]], dtype=float32)>

In [56]:
sparse_delta = tf.IndexedSlices(values=[[1., 2., 3.], [4., 5., 6.]],
                                indices=[1, 0])
v.scatter_update(sparse_delta)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[4., 5., 6.],
       [1., 2., 3.]], dtype=float32)>

In [57]:
sparse_delta = tf.IndexedSlices(values=[[1., 2., 3.], [4., 5., 6.]],
                                indices=[0, 1])
v.scatter_update(sparse_delta)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

##Other Data Structures